# Simulation Basics

These are the basic components we will use for our simulations. This notebook goes over how they work.

In [1]:
import numpy as np

from auctionbets.events import Event
from auctionbets.agents import Agent

## Agents

Agents represent people who make or lay bets. An agent just has their capital, which is the total stake of money they have left, and a list of ids for their open bets (bets they have not yet collected for). 

In [2]:
agent_1 = Agent(starting_capital=100.0)
agent_2 = Agent(starting_capital=100.0)

print(agent_1)
print(agent_2)

**Agent 0**
  Capital: 100.00
  Open bets:
**Agent 1**
  Capital: 100.00
  Open bets:


## Events

Events represent specific sporting events with a binary outcome (win or lose). These are the events that agents can bet on. 

Each event has a "true probability" of happening. This is unknown to the agents. 

In [3]:
event_1 = Event(true_probability=0.5)
print(event_1)

**Event 2**
  True Probability: 0.5
  Bettor Odds: unset
  Layer Odds: unset
  All bets:


Now agents can either bet or lay an event. 

If they bet the event, then they are betting the outcome happens. If it happens, they get their stake back, plus the odds * their stake. 

If the lay the event, they are taking the opposite side of that bet. Therefore, they lose if the outcome happens. If the outcome happens, they lose their money.

Here agent 1 bets on the event, and agent 2 lays it.

In [4]:
bet_1 = agent_1.bet(min_odds=1.5, stake=10.0, event=event_1)
bet_2 = agent_2.lay(max_odds=2.0, stake=30.0, event=event_1)

print(bet_1)
print(bet_2)

Bet(limit_odds=1.5, stake=10.0, agent_idx=0, event_idx=2)
Lay(limit_odds=2.0, stake=30.0, agent_idx=1, event_idx=2)


Now the event and the agents both have those bets associated with them:

In [5]:
print(agent_1)
print(agent_2)
print(event_1)

**Agent 0**
  Capital: 90.00
  Open bets:
    Bet(limit_odds=1.5, stake=10.0, agent_idx=0, event_idx=2)
**Agent 1**
  Capital: 70.00
  Open bets:
    Lay(limit_odds=2.0, stake=30.0, agent_idx=1, event_idx=2)
**Event 2**
  True Probability: 0.5
  Bettor Odds: unset
  Layer Odds: unset
  All bets:
    Bet(limit_odds=1.5, stake=10.0, agent_idx=0, event_idx=2) (unmatched)
    Lay(limit_odds=2.0, stake=30.0, agent_idx=1, event_idx=2) (unmatched)


## Running an event

An event needs to first be matched, which is where we run the double auction mechanism and match up all possible bets. Those bets that cannot be matched get their stake returned.

In [13]:
event = Event(true_probability=0.5)

n_agents = 20
agents_pool = [Agent(starting_capital=0) for _ in range(n_agents)]

is_lay_lst = np.random.choice([True, False], n_agents)
stake_lst = np.random.rand(n_agents) * (20.0 - 10.0) + 10.0
odds_lst = np.random.rand(n_agents) + 0.5

for agent, is_lay, stake, odds in zip(agents_pool, is_lay_lst, stake_lst, odds_lst):
    if is_lay:
        agent.lay(event=event, max_odds=odds + 0.1, stake=stake)
    else:
        agent.bet(event=event, min_odds=odds - 0.1, stake=stake)

print(event)

**Event 45**
  True Probability: 0.5
  Bettor Odds: unset
  Layer Odds: unset
  All bets:
    Bet(limit_odds=1.2706045507326658, stake=10.77744296403749, agent_idx=49, event_idx=45) (unmatched)
    Bet(limit_odds=0.5072479011748636, stake=12.401031016640045, agent_idx=50, event_idx=45) (unmatched)
    Bet(limit_odds=0.8868899480113486, stake=15.176645904022724, agent_idx=52, event_idx=45) (unmatched)
    Bet(limit_odds=0.6827579883004777, stake=16.48887181211879, agent_idx=57, event_idx=45) (unmatched)
    Bet(limit_odds=0.5739091613626182, stake=12.407944541911249, agent_idx=60, event_idx=45) (unmatched)
    Bet(limit_odds=1.0065463252212417, stake=11.7104315961871, agent_idx=65, event_idx=45) (unmatched)
    Lay(limit_odds=1.1002033213816844, stake=11.310852577857315, agent_idx=46, event_idx=45) (unmatched)
    Lay(limit_odds=0.6689829784898615, stake=14.088879566424044, agent_idx=47, event_idx=45) (unmatched)
    Lay(limit_odds=1.2969111462504617, stake=15.788598066059686, agent_idx

In [14]:
event.match_bets()


New round
total_stake_bet = 0.0
total_stake_layed = 0.0
current_bet_odds = 0.5072479011748636
current_lay_odds = 1.549610694147199
Lay Needed
next_lay_odds = 1.4739931692978683
Lay matched

New round
total_stake_bet = 0.0
total_stake_layed = 17.142485053270843
current_bet_odds = 0.5072479011748636
current_lay_odds = 1.4739931692978683
Bet Needed
Bet matched

New round
total_stake_bet = 12.401031016640045
total_stake_layed = 17.142485053270843
current_bet_odds = 0.5739091613626182
current_lay_odds = 1.4739931692978683
Bet Needed
Bet matched

New round
total_stake_bet = 24.808975558551296
total_stake_layed = 17.142485053270843
current_bet_odds = 0.6827579883004777
current_lay_odds = 1.4739931692978683
Lay Needed
next_lay_odds = 1.3674562590843191
Lay matched

New round
total_stake_bet = 24.808975558551296
total_stake_layed = 27.609798588062166
current_bet_odds = 0.6827579883004777
current_lay_odds = 1.3674562590843191
Bet Needed
Bet matched

New round
total_stake_bet = 41.29784737067008

In [15]:
print(event)

**Event 45**
  True Probability: 0.5
  Bettor Odds: unset
  Layer Odds: unset
  All bets:
    Bet(limit_odds=0.5072479011748636, stake=12.401031016640045, agent_idx=50, event_idx=45) (matched)
    Bet(limit_odds=0.5739091613626182, stake=12.407944541911249, agent_idx=60, event_idx=45) (matched)
    Bet(limit_odds=0.6827579883004777, stake=16.48887181211879, agent_idx=57, event_idx=45) (matched)
    Bet(limit_odds=0.8868899480113486, stake=15.176645904022724, agent_idx=52, event_idx=45) (matched)
    Bet(limit_odds=1.0065463252212417, stake=11.7104315961871, agent_idx=65, event_idx=45) (unmatched)
    Bet(limit_odds=1.2706045507326658, stake=10.77744296403749, agent_idx=49, event_idx=45) (unmatched)
    Lay(limit_odds=1.549610694147199, stake=17.142485053270843, agent_idx=62, event_idx=45) (matched)
    Lay(limit_odds=1.4739931692978683, stake=10.467313534791323, agent_idx=64, event_idx=45) (matched)
    Lay(limit_odds=1.3674562590843191, stake=18.928040563308763, agent_idx=63, event_id